In [1]:
import psycopg2 as pg2
import pandas as pd
import numpy as np

In [2]:
# Initializing connection to the cleaned database

try:
    conn = pg2.connect(database='dvdrental_cleaned', user='postgres', password='root')
    cur = conn.cursor()
except Exception as error:
    print("Error while connecting to PostgreSQL", error)

In [129]:
# Creating a dataframe containing information about the films and film categories

movies = pd.read_sql('''SELECT f.film_id, f.title, f.release_year, f.replacement_cost, f.rental_rate, f.length, f.rating, 
                    c.name as category, l.name as language
					FROM film_cleaned f
                     INNER JOIN film_category_cleaned fc
                     ON f.film_id = fc.film_id
                     INNER JOIN category_cleaned c
                     ON fc.category_id = c.category_id
                     INNER JOIN language_cleaned l
                     ON l.language_id = f.language_id
                     ;''', conn)

C:\Users\tepip\AppData\Local\Temp\ipykernel_11828\2379716025.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies = pd.read_sql('''SELECT f.film_id, f.title, f.release_year, f. replacement_cost, f.rental_rate, f.length, f.rating,


In [130]:
movies.shape

(1000, 9)

In [131]:
movies.head()

,film_id,title,release_year,replacement_cost,rental_rate,length,rating,category,language
0,1,Academy Dinosaur,2006,20.99,0.99,86,PG,Documentary,English
1,2,Ace Goldfinger,2006,12.99,4.99,48,G,Horror,English
2,3,Adaptation Holes,2006,18.99,2.99,50,NC-17,Documentary,English
3,4,Affair Prejudice,2006,26.99,2.99,117,G,Horror,English
4,5,African Egg,2006,22.99,2.99,130,G,Family,English


In [132]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           1000 non-null   int64  
 1   title             1000 non-null   object 
 2   release_year      1000 non-null   int64  
 3   replacement_cost  1000 non-null   float64
 4   rental_rate       1000 non-null   float64
 5   length            1000 non-null   int64  
 6   rating            1000 non-null   object 
 7   category          1000 non-null   object 
 8   language          1000 non-null   object 
dtypes: float64(2), int64(3), object(4)
memory usage: 70.4+ KB


In [137]:
movies['film_id'].nunique()

1000

In [138]:
movies.isnull().sum()

film_id             0
title               0
release_year        0
replacement_cost    0
rental_rate         0
length              0
rating              0
category            0
language            0
dtype: int64

In [139]:
# Getting the descriptive statistics for the columns containing numeric values

movies[['rental_rate', 'length', 'replacement_cost']].describe()

,rental_rate,length,replacement_cost
count,1000.000000,1000.000000,1000.000000
mean,2.980000,115.272000,19.984000
std,1.646393,40.426332,6.050833
min,0.990000,46.000000,9.990000
25%,0.990000,80.000000,14.990000
50%,2.990000,114.000000,19.990000
75%,4.990000,149.250000,24.990000
max,4.990000,185.000000,29.990000


In [140]:
# Getting the value count for the different film ratings

movies['rating'].value_counts().sort_values(ascending=False)

rating
PG-13    223
NC-17    210
R        195
PG       194
G        178
Name: count, dtype: int64

In [141]:
# Getting the value count for the different languages

movies['language'].value_counts().sort_values(ascending=False)

language
English                 1000
Name: count, dtype: int64

In [142]:
# Getting the value count for the different film genres

movies['category'].value_counts().sort_values(ascending=False)

category
Sports         74
Foreign        73
Family         69
Documentary    68
Animation      66
Action         64
New            63
Drama          62
Sci-Fi         61
Games          61
Children       60
Comedy         58
Classics       57
Travel         57
Horror         56
Music          51
Name: count, dtype: int64

In [143]:
# Getting the value counts for the movies with the different rental rates

movies['rental_rate'].value_counts().sort_values(ascending=False)

rental_rate
0.99    341
4.99    336
2.99    323
Name: count, dtype: int64

In [144]:
# Getting the value counts for the movies with the different replacement costs

movies['replacement_cost'].value_counts().sort_values(ascending=False)

replacement_cost
20.99    57
22.99    55
12.99    55
21.99    55
13.99    55
27.99    53
29.99    53
14.99    51
19.99    50
11.99    49
10.99    49
17.99    47
26.99    46
23.99    45
25.99    43
18.99    42
28.99    41
9.99     41
24.99    38
16.99    38
15.99    37
Name: count, dtype: int64

In [145]:
# Getting the value counts for the movies with different length

movies_grouped_by_length = movies.groupby('length').count()
movies_grouped_by_length['film_id'].sort_index(ascending=False)

length
185    10
184     8
183     5
182     6
181    10
       ..
50      9
49      5
48     11
47      7
46      5
Name: film_id, Length: 140, dtype: int64

In [151]:
# Getting the titles of the 10 longest movies

movies[['title', 'length']].sort_values(by='length', ascending=False).head(10)

,title,length
348,Gangs Pride,185
181,Control Anthem,185
608,Muscle Bright,185
425,Home Pity,185
816,Soldiers Evolution,185
211,Darn Forrester,185
990,Worst Banger,185
871,Sweet Brotherhood,185
689,Pond Seattle,185
140,Chicago North,185


In [146]:
# Getting the value count for the distribution of the release years

movies['release_year'].value_counts().sort_values(ascending=False)

release_year
2006    1000
Name: count, dtype: int64

In [147]:
# Getting the count of each rating in the different film categories

movies_grouped_by_category = movies.groupby('category')
movies_grouped_by_category['rating'].value_counts().head(20)

category   rating
Action     G         18
           R         14
           NC-17     12
           PG-13     11
           PG         9
Animation  PG-13     19
           NC-17     15
           G         13
           PG        11
           R          8
Children   PG        15
           PG-13     14
           NC-17     12
           G         10
           R          9
Classics   R         14
           PG-13     13
           G         11
           PG        10
           NC-17      9
Name: count, dtype: int64

In [30]:
# Creating a dataframe containing information about rentals, payment amount and customers

rentals = pd.read_sql('''SELECT r.rental_id, r.rental_date, p.payment_id, p.amount, p.payment_date,
                      c.customer_id, c.full_name, c.active
                      FROM rental_cleaned r
                      INNER JOIN payment_cleaned p
                      ON r.rental_id = p.rental_id
                      INNER JOIN customer_cleaned c
                      ON c.customer_id = r.customer_ID;''', conn)

C:\Users\tepip\AppData\Local\Temp\ipykernel_11828\3783422505.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rentals = pd.read_sql('''SELECT r.rental_id, r.rental_date, p.payment_id, p.amount, p.payment_date,


In [31]:
rentals.shape

(14596, 8)

In [32]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14596 entries, 0 to 14595
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   rental_id     14596 non-null  int64         
 1   rental_date   14596 non-null  datetime64[ns]
 2   payment_id    14596 non-null  int64         
 3   amount        14596 non-null  float64       
 4   payment_date  14596 non-null  datetime64[ns]
 5   customer_id   14596 non-null  int64         
 6   full_name     14596 non-null  object        
 7   active        14596 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(4), object(1)
memory usage: 912.4+ KB


In [33]:
rentals.head()

,rental_id,rental_date,payment_id,amount,payment_date,customer_id,full_name,active
0,1520,2005-06-15 23:57:20,17503,7.99,2007-02-15 22:25:46.996577,341,Peter Menard,1
1,1778,2005-06-16 18:54:48,17504,1.99,2007-02-16 17:23:14.996577,341,Peter Menard,1
2,1849,2005-06-17 00:13:19,17505,7.99,2007-02-16 22:41:45.996577,341,Peter Menard,1
3,2829,2005-06-19 21:11:30,17506,2.99,2007-02-19 19:39:56.996577,341,Peter Menard,1
4,3130,2005-06-20 19:03:22,17507,7.99,2007-02-20 17:31:48.996577,341,Peter Menard,1


In [34]:
rentals.isnull().sum()

rental_id       0
rental_date     0
payment_id      0
amount          0
payment_date    0
customer_id     0
full_name       0
active          0
dtype: int64

In [152]:
rentals['rental_id'].nunique()

14592

In [153]:
rentals['payment_id'].nunique()

14596

In [35]:
# Getting the descriptive statistics for the columns containing number values

rentals['amount'].describe()

count    14596.000000
mean         4.200606
std          2.368946
min          0.000000
25%          2.990000
50%          3.990000
75%          4.990000
max         11.990000
Name: amount, dtype: float64

In [36]:
# Getting the number of unique customers who placed a rental

rentals['customer_id'].nunique()

599

In [89]:
# Getting the number of payments grouped by payment amount

rentals_grouped_by_amount = rentals.groupby('amount').count().sort_index(ascending=True)
rentals_grouped_by_amount['rental_id']


amount
0.00       24
0.99     2720
1.98        1
1.99      580
2.99     3233
3.98        8
3.99      988
4.99     3424
5.98        7
5.99     1188
6.99     1017
7.98        5
7.99      621
8.97        1
8.99      438
9.98        1
9.99      233
10.99      99
11.99       8
Name: rental_id, dtype: int64

In [92]:
# Getting the timespan between the first and the last rental date

rental_date = 'Between ' + str(rentals['rental_date'].dt.date.min()) + ' and ' + str(rentals['rental_date'].dt.date.max())
rental_date

'Between 2005-06-14 and 2006-02-14'

In [93]:
# Getting the timespan between the first and the last payment date

payment_date = 'Between ' + str(rentals['payment_date'].dt.date.min()) + ' and ' + str(rentals['payment_date'].dt.date.max())
payment_date

'Between 2007-02-14 and 2007-05-14'

In [20]:
# Creating a dataframe containing customer details

customers = pd.read_sql('''SELECT c.customer_id, c.full_name, c.email, c.address_id, c.activebool, c.active, ci.city, co.country
                        FROM customer_cleaned c
                        INNER JOIN address_cleaned a
                        ON a.address_id = c.address_id
                        INNER JOIN city_cleaned ci
                        ON ci.city_id = a.city_id
                        INNER JOIN country_cleaned co
                        ON co.country_id = ci.country_id
                        ;''', conn)

C:\Users\tepip\AppData\Local\Temp\ipykernel_11828\4159386412.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers = pd.read_sql('''SELECT c.customer_id, c.full_name, c.email, c.address_id, c.activebool, c.active, ci.city, co.country


In [21]:
customers.head()

,customer_id,full_name,email,address_id,activebool,active,city,country
0,524,Jared Ely,jared.ely@sakilacustomer.org,530,True,1,Purwakarta,Indonesia
1,1,Mary Smith,mary.smith@sakilacustomer.org,5,True,1,Sasebo,Japan
2,2,Patricia Johnson,patricia.johnson@sakilacustomer.org,6,True,1,San Bernardino,United States
3,3,Linda Williams,linda.williams@sakilacustomer.org,7,True,1,Athenai,Greece
4,4,Barbara Jones,barbara.jones@sakilacustomer.org,8,True,1,Myingyan,Myanmar


In [154]:
customers['customer_id'].nunique()

599

In [22]:
customers.shape

(599, 8)

In [23]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  599 non-null    int64 
 1   full_name    599 non-null    object
 2   email        599 non-null    object
 3   address_id   599 non-null    int64 
 4   activebool   599 non-null    bool  
 5   active       599 non-null    int64 
 6   city         599 non-null    object
 7   country      599 non-null    object
dtypes: bool(1), int64(3), object(4)
memory usage: 33.5+ KB


In [24]:
customers.isnull().sum()

customer_id    0
full_name      0
email          0
address_id     0
activebool     0
active         0
city           0
country        0
dtype: int64

In [25]:
# Getting the value count for the active and inactive customers

customers['active'].value_counts().sort_values(ascending=False)

active
1    584
0     15
Name: count, dtype: int64

In [79]:
# Getting the percentage of the inactive customers/total number of customers

ratio = (customers['active'].value_counts()[0]/customers.shape[0]) * 100
round(ratio, 2)

2.5

In [26]:
# Getting the value count for how many customers live in each city

customers['city'].value_counts().sort_values(ascending=False)

city
London          2
Aurora          2
Witten          1
Kurashiki       1
Kaduna          1
               ..
Szkesfehrvr     1
Garden Grove    1
Jining          1
Jinzhou         1
Tieli           1
Name: count, Length: 597, dtype: int64

In [27]:
# Getting the value count for how many customers live in each country

customers['country'].value_counts().sort_values(ascending=False)

country
India            60
China            53
United States    36
Japan            31
Mexico           30
                 ..
Chad              1
Lithuania         1
Iraq              1
Bahrain           1
Hungary           1
Name: count, Length: 108, dtype: int64

In [157]:
# Creating a dataframe about films and actors

actors = pd.read_sql('''SELECT a.actor_id, a.full_name, f.film_id, f.title
                     FROM actor_cleaned a
                     INNER JOIN film_actor_cleaned fa
                     ON a.actor_id = fa.actor_id
                     INNER JOIN film_cleaned f
                     ON f.film_id = fa.film_id''', conn)

C:\Users\tepip\AppData\Local\Temp\ipykernel_11828\3921919229.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  actors = pd.read_sql('''SELECT a.actor_id, a.full_name, f.film_id, f.title


In [165]:
actors.head(10)

,actor_id,full_name,film_id,title
0,1,Penelope Guiness,1,Academy Dinosaur
1,1,Penelope Guiness,23,Anaconda Confessions
2,1,Penelope Guiness,25,Angels Life
3,1,Penelope Guiness,106,Bulworth Commandments
4,1,Penelope Guiness,140,Cheaper Clyde
5,1,Penelope Guiness,166,Color Philadelphia
6,1,Penelope Guiness,277,Elephant Trojan
7,1,Penelope Guiness,361,Gleaming Jawbreaker
8,1,Penelope Guiness,438,Human Graffiti
9,1,Penelope Guiness,499,King Evolution


In [160]:
actors['actor_id'].nunique()

200

In [161]:
actors['film_id'].nunique()

997

In [164]:
actors.isnull().sum()

actor_id     0
full_name    0
film_id      0
title        0
dtype: int64

In [166]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5462 entries, 0 to 5461
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   actor_id   5462 non-null   int64 
 1   full_name  5462 non-null   object
 2   film_id    5462 non-null   int64 
 3   title      5462 non-null   object
dtypes: int64(2), object(2)
memory usage: 170.8+ KB


In [176]:
# Getting the top 5 actors who had the most appearances in the movies

group_by_actor = actors.groupby(by='full_name').count()
group_by_actor.sort_values(by='film_id', ascending=False)['film_id'].head(5)


full_name
Susan Davis       54
Gina Degeneres    42
Walter Torn       41
Mary Keitel       40
Matthew Carrey    39
Name: film_id, dtype: int64

In [182]:
# Getting the top 7 which had the most actor appearances in them

group_by_film = actors.groupby(by='title').count()
group_by_film.sort_values(by='actor_id', ascending=False)['actor_id'].head(7)

title
Lambs Cincinatti     15
Crazy Home           13
Random Go            13
Boondock Ballroom    13
Chitty Lock          13
Mummy Creatures      13
Dracula Crystal      13
Name: actor_id, dtype: int64

In [215]:
# Getting the titles of the movies without any actors from the database appearing in them

movies_without_actors = pd.read_sql('''SELECT f.film_id, f.title, fa.film_id
                                    FROM film_cleaned f
                                    LEFT JOIN film_actor_cleaned fa
                                    ON f.film_id = fa.film_id''', conn)

C:\Users\tepip\AppData\Local\Temp\ipykernel_11828\4098418685.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies_without_actors = pd.read_sql('''SELECT f.film_id, f.title, fa.film_id


In [216]:
nan_mask = pd.isna(movies_without_actors).any(axis=1)
movies_without_actors = movies_without_actors[nan_mask]

In [217]:
movies_without_actors['title']

5462    Slacker Liaisons
5463         Flight Lies
5464    Drumline Cyclone
Name: title, dtype: object